In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/Users/finn/Documents/GitHub/data/e4e8/migration_pairs.xlsx', sheet_name='Table 1', skiprows=10)
df = df[
    ['Region, development group, country or area of destination',
     'Region, development group, country or area of origin',
     'Location code of origin',
     'Location code of destination',
    1990,
    1995,
    2000,
    2005,
    2010,
    2015,
    2020,
     ]]
df = df.rename(columns={
    'Region, development group, country or area of destination': 'destination',
    'Region, development group, country or area of origin': 'origin',
    'Location code of origin': 'origin_code',
    'Location code of destination': 'destination_code',
    1990: '1990',
    1995: '1995',
    2000: '2000',
    2005: '2005',
    2010: '2010',
    2015: '2015',
    2020: '2020'})
df

In [ ]:
df = df.melt(id_vars=['destination', 'origin', 'origin_code', 'destination_code'], var_name='year', value_name='value')

In [ ]:
df.index = df['origin_code'].astype(str) + '_' + df['destination_code'].astype(str) + "_" + df['year'].astype(str)

In [ ]:
# select by index
df.loc['900_900_1990']

In [18]:
df.sort_values(by='net_flow', ascending=False)

,origin,destination,origin_code,destination_code,year,value,net_flow
840_484_2020,United States of America*,Mexico,840,484,2020,799248,10053857
784_356_2020,United Arab Emirates,India,784,356,2020,38470,3432830
840_356_2020,United States of America*,India,840,356,2020,32867,2690897
682_356_2020,Saudi Arabia,India,682,356,2020,14448,2487889
356_50_2020,India,Bangladesh,356,50,2020,33762,2454709
...,...,...,...,...,...,...,...
716_850_2020,None,None,716,850,2020,None,None
716_876_2020,None,None,716,876,2020,None,None
716_732_2020,None,None,716,732,2020,None,None
716_887_2020,None,None,716,887,2020,None,None


In [ ]:
def netflow(row):
    if row.origin_code == row.destination_code:
        return None
    opp_row = f'{row.destination_code}_{row.origin_code}_{row.year}'
    if opp_row not in df.index:
        return None
    return df.loc[opp_row, 'value']-row.value


In [ ]:
df['net_flow'] = df.apply(netflow, axis=1)

In [ ]:
#df = pd.read_json('/Users/finn/Documents/GitHub/data/e4e8/migration_pairs.json')

In [ ]:
df.net_flow = df.net_flow.astype(int, errors='ignore')

In [ ]:
# Replace lists with None
df.net_flow = df.net_flow.apply(lambda x: None if isinstance(x, list) else x)

In [ ]:
df = df[df.year == '2020']

In [ ]:
df = df[[ 'origin', 'destination','origin_code', 'destination_code', 'year', 'value', 'net_flow']]

Restricting to only countries:

In [ ]:
iso_df = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")

In [ ]:
df = df[df.origin_code.isin(iso_df['country-code'])]
df = df[df.destination_code.isin(iso_df['country-code'])]

In [ ]:
df

In [ ]:
for i, origin_country in enumerate(iso_df['country-code']):
    print(i)
    for destination_country in iso_df['country-code']:
        if f'{origin_country}_{destination_country}_2020' not in df.index:
            df.loc[f'{origin_country}_{destination_country}_2020'] = [None, None, origin_country, destination_country, '2020', None, None]


iso_df['country-code']

In [ ]:
df

In [ ]:
df

In [24]:
df.to_json('/Users/finn/Documents/GitHub/data/e4e8/migration_pairs.json', orient='records')

In [ ]:
df.query('origin_code == 900')

In [27]:
df.query('origin_code == 826').sort_values(by='net_flow', ascending=True)

,origin,destination,origin_code,destination_code,year,value,net_flow
826_36_2020,United Kingdom*,Australia*,826,36,2020,1285149,-1144914
826_840_2020,United Kingdom*,United States of America*,826,840,2020,673667,-502896
826_124_2020,United Kingdom*,Canada,826,124,2020,537504,-436966
826_554_2020,United Kingdom*,New Zealand*,826,554,2020,286746,-207268
826_724_2020,United Kingdom*,Spain*,826,724,2020,303193,-152301
...,...,...,...,...,...,...,...
826_860_2020,None,None,826,860,2020,None,None
826_850_2020,None,None,826,850,2020,None,None
826_876_2020,None,None,826,876,2020,None,None
826_732_2020,None,None,826,732,2020,None,None
